In [1]:
import numpy as np
import pandas as pd
import folium
import json

In [2]:
df = pd.read_csv('../simulation_data/simulation_data.csv')
df.rename(columns={'median_PM10': 'PM10'}, inplace=True)
df['index'] = df.index # needed for folium choropleth mapping

In [3]:
# size of grid fields in °
step = 0.0001

# returns GeoJSON information of a single square
def get_square_json(index, lat, lon, step):
    return {
          "type": "Feature",
          "id": index,
          "properties": {
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [lon, lat],
                [lon, lat + step],
                [lon + step, lat + step],
                [lon + step, lat],
                [lon, lat],
              ]]}}

# create json with all squares in grid
geo_json = "{\"type\":\"FeatureCollection\", \"features\": ["
for index, row in df.iterrows():
    if index != 0:
        geo_json += ','
    square_dict = get_square_json(index, row['latitude_index'], row['longitude_index'], step)
    geo_json += json.dumps(square_dict)
geo_json += "]}"

In [4]:
m = folium.Map(location=[df['latitude_index'][0], df['longitude_index'][0]], control_scale = True, zoom_start=16)

overlays = ['PM10']
featuregroups = np.empty(len(overlays), dtype=object)

bins = [0, 3, 6, 9, 12, 15, 18, 21, 25]

choropleth_params = {'geo_data': geo_json,
                     'key_on': 'feature.id',
                     'fill_color': 'YlOrRd',
                     'nan_fill_color': 'White',
                     'fill_opacity': 0.7,
                     'line_opacity': 0.5,
                     'legend_name':'Mean PM2.5 in µg/m³ (of all data points in grid field)',
                     'bins': bins,
                     'highlight': True,
                     'reset': True
                    }

for idx, feature in enumerate(overlays):
    featuregroups[idx] = folium.FeatureGroup(name=overlays[idx],overlay=False).add_to(m)
    
    # make sure that data column does not contain nan values
    choropleth_data = df.dropna(subset=[feature]) 
    
    choropleth_params['data'] = choropleth_data
    choropleth_params['columns'] = ['index', feature]
    
    choropleth = folium.Choropleth(**choropleth_params)
    choropleth.geojson.add_to(featuregroups[idx])

# add legend
legend = list(choropleth._children.values())[1]
legend.add_to(m)
    
#Add layer control to the map
folium.TileLayer('openstreetmap',overlay=True,name="View in openstreetmap").add_to(m)
folium.TileLayer('cartodbpositron',overlay=True,name="View in Light Mode", show=False).add_to(m)
folium.LayerControl(position='bottomright', collapsed=False).add_to(m)
m